### Task 3 : Get the followers of a given twitter user from your acquired data set.

#### Imports

In [4]:
# Imports
import pandas as pd # Pandas for dataframes
import requests # Requests for API calls to the followers API in online mode
from IPython.display import display # Display dataframes in Jupyter in conditional statements

#### Settings

In [5]:
# Brearer token required for the online mode of this task
# NOT REQUIRED FOR THE OFFLINE MODE
bearer_token = ""

#### Utility functions

In [6]:
# Return the user id of a user with a given username
def user_id_by_username(username, datasets):

    # Remove the @ symbol, if it exists
    username = username.replace("@", "")

    # Look for the user in the datasets
    for dataset in datasets: # For each dataset

        # Load the authors
        authors_df = pd.read_csv("data/" + dataset + "/authors.csv")

        # Check if the username exists in the dataset using the username column
        author = authors_df[authors_df['username'] == username]

        # If the username exists, return the user id
        if len(author) > 0:
            return author['id'].values[0]
    print("The user " + username + " does not exist in the datasets.")
    return None

#### Main functions

In [7]:
def get_followers_online(user_id, max_results):
    
    #url = "https://api.twitter.com/2/users/15237599045232271/followers?user.fields=id,created_at,name,username,protected,verified,profile_image_url,location,url,description,pinned_tweet_id,public_metrics"
    url = "https://api.twitter.com/2/users/" + str(user_id) + "/followers?max_results=" + str(max_results) + "&user.fields=id,created_at,name,username,protected,verified,profile_image_url,location,url,description,pinned_tweet_id,public_metrics"
    res = requests.get(url, headers={"Authorization": "Bearer " + bearer_token})

    # Check if there is an error
    if (res.json().get("errors")):
        print("Error:", res.json()["errors"])
        return None

    # Convert the response to a dataframe
    followers_df = pd.DataFrame(res.json()["data"])

    # Save the dataframe to a csv file
    followers_df.to_csv("data/followers/" + str(user_id) + ".csv", index=False)

    # Return the dataframe
    return followers_df

def get_followers_offline(user_id, max_results):

    # Try to load the dataframe from the csv file
    try:

        # Load the followers dataframe
        followers_df = pd.read_csv("data/followers/" + str(user_id) + ".csv")
    
    # If the file does not exist, return None
    except FileNotFoundError:
        print('The followers of this user are not in the dataset')
        return None


    # Only return at most max_results results
    if len(followers_df) > max_results:
        followers_df = followers_df.iloc[:max_results]

    # Return the dataframe
    return followers_df

#### Exemple usage : online

In [8]:
datasets = ["Paris"]
id = user_id_by_username("@ParisAntan", datasets)

if (id is not None):
    followers_df = get_followers_online(id, 1000)[["id", "username", "name"]]
    display(followers_df)

,id,username,name
0,1536116954854920192,romamcallister,dani
1,1290972481872855040,Sissi785,Marie le Doucen Perro
2,1108870543095279616,BikeTignolles,BikeTignolles
3,1535547331956842499,SHAParis8_17,Société Historique et Archéologique Paris 8e &...
4,1975358845,tnnblstn,Étienne™
...,...,...,...
638,1357009623815970818,LecomteBenedik1,Le comte Benedikte
639,390751370,JCTrehan,Jean-Charles Tréhan
640,1162790363532316677,MonumentumFr,Monumentum
641,250835184,artispat,Artisans du Patrimoine


#### Exemple usage : offline

In [9]:
datasets = ["Paris"]
id = user_id_by_username("@ParisAntan", datasets)

if (id is not None):
    followers_df = get_followers_offline(id, 348)[["id", "username", "name"]]
    display(followers_df)

,id,username,name
0,1536116954854920192,romamcallister,dani
1,1290972481872855040,Sissi785,Marie le Doucen Perro
2,1108870543095279616,BikeTignolles,BikeTignolles
3,1535547331956842499,SHAParis8_17,Société Historique et Archéologique Paris 8e &...
4,1975358845,tnnblstn,Étienne™
...,...,...,...
343,1429146334104281091,deaky_fr,DeakyEnTouraine
344,1241656726023278592,CanPMarat,Z- ЗА ПОБЕДЯ ☭🦺
345,254496830,clement1733,Clémentª
346,772726452412354561,kangouroublanc1,KgrB
